In [4]:
import os
from autogen import ConversableAgent

In [6]:
!pip install markdownify

In [7]:
from autogen.browser_utils import SimpleTextBrowser

In [8]:
browser = SimpleTextBrowser()

In [15]:
browser.address

'https://www.ceat.com/investors/annual-reports.html'

In [92]:
# !pip install selenium

In [80]:
os.listdir('./chromedriver_mac_arm64')

['chromedriver', 'LICENSE.chromedriver']

## Download Reports from Ceat Website

In [91]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
import time

# Set up the WebDriver
download_dir = os.getcwd() + "/ceat"  # Change this to your desired download folder
print(download_dir)
os.makedirs(download_dir, exist_ok=True)

# Set Chrome preferences to automate downloads
options = Options()
prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,  # To automatically download the PDF
        # "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True  # It will not open the PDF in the browser
    }
options.add_experimental_option("prefs", prefs)
# options.add_argument("--headless")

    # Initialize WebDriver with options
    # service = Service(executable_path="/path/to/chromedriver")  # Update the path to where you've downloaded chromedriver


service = Service(executable_path="./chromedriver_mac_arm64/chromedriver")
driver = webdriver.Chrome(options=options)
print(driver)

# URL of the webpage
url = 'https://www.ceat.com/investors/annual-reports.html'

# Navigate to the URL
driver.get(url)

# Extract the specific component
# Replace 'your-css-selector' with the actual CSS selector for your component
elements = driver.find_elements(By.CSS_SELECTOR, 'a.btn-icon')

report_urls = [element.get_attribute('href') for element in elements]
print("report-urls", report_urls)

if report_urls:
    for report_url in report_urls:
        driver.get(report_url)
        WebDriverWait(driver, 10)
    

# download_pdf(driver, report_urls[0])    # driver.get(report_urls[0])
print("postwait")

time.sleep(60)
driver.quit()

/Users/paruljuniwal/learning_development_Interview/product_suite/ai/fin-robo/ceat
<selenium.webdriver.chrome.webdriver.WebDriver (session="1777383c826f161e4017f466c58b944c")>
report-urls ['https://www.ceat.com/content/dam/ceat/pdf/Annual_Reports/CEAT-Limited-Integrated-Annual-Report-FY23.pdf', 'https://www.ceat.com/content/dam/ceat/pdf/CEAT%20AR%202021-22_Spread-27.6.22.pdf', 'https://www.ceat.com/content/dam/ceat/pdf/Annual_Reports/Ceat-Limited-Integrated-Annual-Report-FY21.pdf', 'https://www.ceat.com/content/dam/ceat/pdf/Annual_Reports/CEAT%20Limited%20Integrated%20Annual%20Report%20FY20.pdf', 'https://www.ceat.com/content/dam/ceat/pdf/Annual_Reports/CEAT%20Limited%20Annual%20Report%20FY19.pdf', 'https://www.ceat.com/content/dam/ceat/pdf/Annual_Reports/CEAT%20Limited%20Annual%20Report%20FY18.pdf', 'https://www.ceat.com/content/dam/ceat/pdf/Annual_Reports/CEAT%20Limited%20Annual%20Report%20FY17.pdf', 'https://www.ceat.com/content/dam/ceat/pdf/Annual_Reports/CEAT%20Limited%20Annual%20R

## Upload PDF files and convert to structured data

In [105]:
reports = [report for report in os.listdir("./ceat") if report.endswith(".pdf")]

print("length:", len(reports), "::::All reports: ", reports)

length: 8 ::::All reports:  ['CEAT Limited Annual Report FY17.pdf', 'CEAT Limited Annual Report FY16.pdf', 'CEAT Limited Integrated Annual Report FY20.pdf', 'CEAT Limited Annual Report FY18.pdf', 'CEAT Limited Annual Report FY19.pdf', 'CEAT AR 2021-22_Spread-27.6.22.pdf', 'Ceat-Limited-Integrated-Annual-Report-FY21.pdf', 'CEAT-Limited-Integrated-Annual-Report-FY23.pdf']


In [106]:
from langchain_community.document_loaders import PyPDFLoader

In [109]:
reports[0]

'CEAT Limited Annual Report FY17.pdf'

In [112]:
loader = PyPDFLoader(os.path.join(os.getcwd(), "ceat", reports[0]))
pages = loader.load_and_split()

In [113]:
len(pages)

400

In [123]:
pages[0].page_content

'TOWARDS A SAFER, \nSMARTER TOMORROW\nAnnual Report 2016-17'

In [ ]:
from pprint import pprint
pprint(pages[1].page_content)

In [124]:
for i in range(0, 30):
    print("*" * 100)
    pprint(pages[i].page_content)
    print("*" * 100)

****************************************************************************************************
'TOWARDS A SAFER, \nSMARTER TOMORROW\nAnnual Report 2016-17'
****************************************************************************************************
****************************************************************************************************
('Corporate Overview Statutory Reports Financial Section\n'
 '28  Management Discussion \n'
 'and Analysis\n'
 '51  Discussion on the Financial \n'
 'Performance\n'
 '64 Notice\n'
 '74  Board’s Report\n'
 '108 Corporate Governance Report\n'
 '128 Business Responsibility Report136  Standalone Financial  \n'
 'Statements\n'
 '219  Consolidated Financial \n'
 'Statements\n'
 'Forward-looking statement\n'
 'In this Annual Report, we have disclosed forward looking information to '
 'enable investors to comprehend our prospects and take investment decisions. '
 'This \n'
 'report and other statements - written and oral - that we perio

In [130]:
from autogen import ConversableAgent, AssistantAgent, UserProxyAgent

In [129]:
local_llama2_uncensored = {
    "model": "llama2-uncensored",
    "base_url": "http://localhost:11434/v1/",
    "temperature": 0.9
    # "api_type": "openai"
}
local_llama3 = {
    "model": "llama3",
    "base_url": "http://localhost:11434/v1/",
    "temperature": 0.2,
    "cache_seed": 42,
    # "api_type": "openai"
}

In [229]:
# assistant = AssistantAgent(
#     name="Assistant",
#     system_message="Only use the tools you have been provided with. Reply TERMINATE when the task is done.",
#     llm_config={"config_list": config_list, "cache_seed": None},
# )

# user_proxy = UserProxyAgent(
#     name="user_proxy",
#     is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
#     human_input_mode="TERMINATE",
#     max_consecutive_auto_reply=10,
#     # code_execution_config={"executor": code_executor},
# )

user_proxy = ConversableAgent(
    name="Editor",
    system_message="""You are the admin editor overseeing the analysis. 
    Interact with the financial analyst agent to discuss the analysis. 
    Provide a feedback to the financial analyst's content on the following lines:
    1. A detailed explantion of the points
    2. No NSFW language""",
    code_execution_config=False,
    human_input_mode="ALWAYS"
)


financial_analyst = ConversableAgent(
    name="Equity Analyst",
    system_message="""You’re an Equity analyst with expertise in auto and auto parts. 
    You have analysed annual developement in all tyre companies in India.  
    How do you think work done by CEAT can help to grow its product portfolio, customer delight, growth in overall revenue 
    and profitability?""",
    
    # llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    llm_config={"config_list": [local_llama3]},
    human_input_mode="NEVER",  # Never ask for human input.
)

# Your name is Cathy. You are the most seasoned financial analyst with 
#     lots of expertise in stock market analysis and investment strategies in a company providing portfolio management 
#     and financial consultancy services for High Networth Individuals. 
#     You are tasked to analyse the report provided for a company's financials and write engaging blogpost.
#     You try generate the best blogpost possible for the user's request. 
#     If the user provides critique, respond with a revised version of your previous attempts.""",

In [194]:
def get_page(idx):
    return pages[idx]

In [230]:
# first_query = ""
# for i in pages[10:27]:
#     first_query += i.page_content

# print(first_query)

In [231]:
task = f"""Give me a detailed analysis of the following text from the financial report. 
Wait for me to analyse your analyse and add comments. Revise your analysis as per my comments.

The content is as follows :\n {first_query}
"""

In [196]:
# print(task)

In [232]:
chat = user_proxy.initiate_chat(
        financial_analyst,
        message=task,
        max_turns=3,
        # cache=cache,
    )

Editor (to Equity Analyst):

Give me a detailed analysis of the following text from the financial report. 
Wait for me to analyse your analyse and add comments. Revise your analysis as per my comments.

The content is as follows :
 STATUTORY REPORTS FINANCIAL STATEMENTS
ANNUAL REPORT 2016-1709YEAR IN REVIEW
1. Awards and Recognitions
 CEAT ranked No. 1 in J.D Power 
2017 Study Results in Passenger 
car and Utility vehicle segment for 
new vehicles 
 Halol Plant awarded with the ‘Sword of Honour’ award by British 
Safety Council
 Halol plant received 5 Star rating 
from British Safety Council for 
upgraded safety measures 
implemented at the plant.
 CEAT ranked No. 2 in the “middle weight” category in the India’s 
Fastest Growing Companies issue 
of Business World
 CEAT won the “End to End Customer Solution” category at 6th 
Annual Manufacturing & Supply 
Chain Awards, 2016
2. Enduring OEM partnership
 CEAT was the first source for new vehicles like Hero Achiever, Bajaj 
V12, Honda Navi

KeyboardInterrupt: Interrupted by user

In [166]:
print(len(chat.chat_history))

6


In [214]:
# for i in chat.chat_history:
#     print(i)
#     print("*" * 100)

In [215]:
# print(chat.chat_history[1]["content"])

In [216]:
# print(chat.chat_history[0]["content"])

In [ ]:
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke and then say goodbye.", max_turns=3)

In [ ]:
# return Agent(
#       role='The Best Financial Analyst',
#       goal="""Impress all customers with your financial data 
#       and market trends analysis""",
#       backstory="""The most seasoned financial analyst with 
#       lots of expertise in stock market analysis and investment
#       strategies that is working for a super important customer.""",
#       verbose=True,

In [202]:
print(pages[11].page_content)

CORPORATE OVERVIEW
CEAT LIMITED10KEY PERFORMANCE INDICATORS (CONSOLIDATED) 
EBITDA (` in lacs) EBITDA (%)
FY13
 45,562 9.0712.16 12.18 14.53
11.67 
FY14
   67,186 
FY15
  70,299 
FY16*
 80,088 
FY17*   
   67,541 EBIDTA
Gross Margin(`  in lacs) Gross Margin (%)
FY13
  1,48,947 30.03 34.7538.05 43.2640.59
FY14
  1,89,885
FY15
   2,16,927
FY16*
   2,35,633 
FY17*
   2,32,257 GROSS MARGIN
Net Sales (`  in lacs) Sales Volume(MT)
FY13
  4,96,523
FY14
  5,46,460
FY15
   5,70,259
FY16*
   5,44,718NET SALES/ SALES VOLUME
FY17*
   5,72,233
*  The Company transitioned to IndAS from April 1, 2015 
Thus previous year’s figures are not comparable.
1 Total  production figures are  only  for standalone.
2 EBITDA includes Non Operating income.
FY13
 2,17,365
FY14
 2,42,540
FY15
 2,51,692
FY16
 2,70,040
FY17
 2,98,698 TOTAL PRODUCTION (MT)
  2,21,5432,44,3002,58,668 2,75,573 3,09,540


In [209]:
print(pages[12].page_content)

STATUTORY REPORTS FINANCIAL STATEMENTS
ANNUAL REPORT 2016-1711
PAT (` in lacs) PAT (%)
FY13
  12,019  2.424.965.568.03
6.31
FY14
   27,124  
FY15
    31,718 
FY16*
   43,754 
FY17*
    36,116 PAT
*  The Company transitioned to IndAS from April 1, 2015.    
Thus previous year’s figures are not comparable.
3  PAT is considered after minority interest
FY13
229
FY14
 286
FY15*
424
FY16*
 508
FY17*
597BOOK VALUE PER SHARE (`/share)
FY17*
 FY13
  35.10
FY14
  76.59
FY15*
 84.62
FY16*
 108.17
   89.28 EARNING PER SHARE (`/share)
FY17
 FY13
  4.00 
FY14
    10.00 
FY15
   10.00 
FY16
  11.50 
    11.50 DIVIDEND PER SHARE (`/share)


## Factual Equity Analyst

In [237]:
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="""You are the admin editor overseeing the analysis and query answering. 
    Interact with the financial analyst agent to query factual data the analysis. """,
    code_execution_config=False,
    human_input_mode="NEVER"
)


financial_analyst = AssistantAgent(
    name="Equity Analyst",
    system_message= """You’re an Equity analyst with expertise in auto and auto parts in India. 
    You have analysed annual developement in all tyre companies in India. 
    You will be provided with a document delimited by triple backticks ```  and a question. 
    Your task is to answer the question using only the provided document and to cite the passage(s) 
    of the document used to answer the question. If the document does not contain the information needed 
    to answer this question then simply write: "Insufficient information.
    If an answer to the question is provided, it must be annotated with a citation. 
    Use the following format for to cite relevant passages ({"citation": …}). """,
    
    # llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    llm_config={"config_list": [local_llama3]},
    human_input_mode="NEVER",  # Never ask for human input.
)

In [222]:
def create_prompt(page_content="", question_content=""):
    prompt_template = f""" ```{page_content}``` Question: {question_content}"""
    return prompt_template

In [219]:
task1 = f""" ```{pages[12].page_content}``` Question: What is the PAT for the year 2014 and 2015?"""
task2 = f""" ```{pages[12].page_content}``` Question: What is the PAT for the year 2014 and 2015?"""


In [235]:
task2 = create_prompt(pages[12].page_content, "What is the EPS for the year 2017?")
# print(task2)

In [238]:
chat = user_proxy.initiate_chat(
        financial_analyst,
        message=task2,
        max_turns=3,
        # cache=cache,
)

# SImple Chat interfac with llama3